# Stats on providing blog description, profile image

In [3]:
import pandas as pd

# Load 1m data

blog_data_fpath = '/data/icwsm2020_tumblr_identity/icwsm2020_blogs_1m.csv'
# blog_info = pd.read_csv(blog_data_fpath, escapechar='\\', engine='python', encoding='utf8', error_bad_lines=False)
blog_info = pd.read_csv(blog_data_fpath)
print(blog_info.columns)
print(len(blog_info))

Skipping line 119458: Expected 14 fields in line 119458, saw 27
Skipping line 148688: Expected 14 fields in line 148688, saw 27
Skipping line 156335: Expected 14 fields in line 156335, saw 27
Skipping line 230847: Expected 14 fields in line 230847, saw 27
Skipping line 242074: Expected 14 fields in line 242074, saw 27
Skipping line 307609: Expected 14 fields in line 307609, saw 27
Skipping line 360570: Expected 14 fields in line 360570, saw 27
Skipping line 379222: Expected 14 fields in line 379222, saw 27
Skipping line 384288: Expected 14 fields in line 384288, saw 27
Skipping line 472072: Expected 14 fields in line 472072, saw 27
Skipping line 472845: Expected 14 fields in line 472845, saw 27
Skipping line 479855: Expected 14 fields in line 479855, saw 27
Skipping line 503662: Expected 14 fields in line 503662, saw 27
Skipping line 539319: Expected 14 fields in line 539319, saw 27
Skipping line 554024: Expected 14 fields in line 554024, saw 27
Skipping line 662068: Expected 14 fields

Index(['tumblog_id', 'tumblr_blog_name', 'tumblr_blog_title',
       'tumblr_blog_url', 'is_group_blog', 'is_primary', 'created_time_epoch',
       'updated_time_epoch', 'timezone', 'language', 'blog_classifier',
       'reblog_count_2018-06-01/2018-11-30', 'has_blog_description',
       'processed_blog_description'],
      dtype='object')
1000624


In [4]:
# Load profile image info

profile_image_info_fpath = '/projects/icwsm2020_tumblr_identity/output/scrape_info_2019-08-21T0013.txt'

with open(profile_image_info_fpath) as f:
    lines = f.read().splitlines()
    default_start_idx = lines.index('Blog names with default images:')
    image_start_idx = lines.index('Blog names with other images:')
    blognames_default = lines[default_start_idx+1: image_start_idx-1]
    blognames_image = lines[image_start_idx+1:]
    
print(len(blognames_default))
print(len(blognames_image))

283
717


In [7]:
from tqdm import tqdm_notebook as tqdm

def annotate_blog_info(blogname):
    if blogname in blognames_default:
        return 'default'
    elif blogname in blognames_image:
        return 'image'
    else:
        return ''

blog_info['profile_image'] = [annotate_blog_info(name) for name in tqdm(blog_info['tumblr_blog_name'].tolist())]
len(blog_info[blog_info['profile_image']=='default'])

283

In [8]:
with_profile_image = blog_info[blog_info['profile_image']!='']
len(with_profile_image)

1000

In [20]:
# Stats on profile image/blog description
blognames_with_descriptions = set(with_profile_image.loc[with_profile_image['has_blog_description']=='true', 'tumblr_blog_name'])
blognames_with_images = set(with_profile_image.loc[with_profile_image['profile_image']=='image', 'tumblr_blog_name'])

intersect = blognames_with_descriptions.intersection(blognames_with_images)
both = len(intersect)
desc_only = len(blognames_with_descriptions - intersect)
image_only = len(blognames_with_images - intersect)
# print(f"Have descriptions: {len(blognames_with_descriptions)}")
# print(f"Have images: {len(blognames_with_images)}")
print(f"Just descriptions: {desc_only}")
print(f"Just images: {image_only}")
print(f"Both description and image: {both}")

Just descriptions: 120
Just images: 179
Both description and image: 538


In [24]:
# Calculate correlation
from scipy.stats import pearsonr

has_description = [1 if val == 'true' else 0 for val in with_profile_image['has_blog_description'].tolist()]
has_image = [1 if val == 'image' else 0 for val in with_profile_image['profile_image'].tolist()]

pearsonr(has_description, has_image)

(0.3098639330864374, 1.0749575488859195e-23)

In [25]:
blog_info.columns

Index(['tumblog_id', 'tumblr_blog_name', 'tumblr_blog_title',
       'tumblr_blog_url', 'is_group_blog', 'is_primary', 'created_time_epoch',
       'updated_time_epoch', 'timezone', 'language', 'blog_classifier',
       'reblog_count_2018-06-01/2018-11-30', 'has_blog_description',
       'processed_blog_description', 'profile_image'],
      dtype='object')

In [26]:
blog_info.to_csv(blog_data_fpath, index=False)

# Sample 500 blog names from dataset 

In [3]:
import os
import pandas as pd

# Load reblog prediction dataset

data_dirpath = '/usr2/mamille2/tumblr/data/sample1k/'

reblog_fpath = os.path.join(data_dirpath, 'reblogs_descs_annotated', 'reblogs_descs.tsv')
reblogs = pd.read_table(reblog_fpath)
print(len(reblogs))
reblogs.columns

153090


Index(['blog_description_followee', 'blog_name_followee',
       'blog_title_followee', 'blog_url_followee', 'is_group_blog_followee',
       'is_private_followee', 'created_time_epoch_followee',
       'updated_time_epoch_followee', 'timezone_followee', 'language_followee',
       ...
       'interests_terms_followee', 'location_terms_followee',
       'personality type_terms_followee', 'pronouns_terms_followee',
       'relationship status_terms_followee', 'roleplay_terms_followee',
       'roleplay/fandoms_terms_followee', 'sexual orientation_terms_followee',
       'weight_terms_followee', 'zodiac_terms_followee'],
      dtype='object', length=104)

In [6]:
blognames = reblogs['blog_name_followee'].tolist() + reblogs['blog_name_follower'].tolist()
len(blognames)

306180

In [11]:
from tqdm import tqdm_notebook as tqdm

nonreblog_fnames = os.listdir(os.path.join(data_dirpath, 'nonreblogs_descs_match'))
for fname in tqdm(nonreblog_fnames):
    fpath = os.path.join(data_dirpath, 'nonreblogs_descs_match', fname)
    nonreblogs = pd.read_table(fpath)
    blognames += nonreblogs['blog_name_followee'].tolist() + nonreblogs['blog_name_follower'].tolist()
    
len(blognames)

/usr0/home/mamille2/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (6,18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr0/home/mamille2/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (6,13,18,48) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Exception in thread Thread-4:
Traceback (most recent call last):
  File "/usr0/home/mamille2/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr0/home/mamille2/anaconda3/lib/python3.6/site-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/usr0/home/mamille2/anaconda3/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration

/usr0/home/mamille2/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr0/home/mamille2/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (6,9,10,27) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=c

/usr0/home/mamille2/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (9,10,13,27,48) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr0/home/mamille2/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (18,27,48) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr0/home/mamille2/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (6,9,10,13,27,48) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr0/home/mamille2/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (1,6,49) have mixed types. Specify dtype option on import or s

/usr0/home/mamille2/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (6,13,48) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr0/home/mamille2/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (6,18,21,22,27) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr0/home/mamille2/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (1,6,21,22,27,49) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr0/home/mamille2/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (21,22,27) have mixed types. Specify dtype option on import or 

/usr0/home/mamille2/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (1,18,21,22,27,49) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr0/home/mamille2/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (1,2,9,10,13,27,48,49) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr0/home/mamille2/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (1,3,6,9,10,13,27,29,31,42,43,48,49) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr0/home/mamille2/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (1,13,18,48,49) have mixed t

/usr0/home/mamille2/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (9,10,13,18,27,31,48) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr0/home/mamille2/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (1,6,13,18,27,31,48,49) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr0/home/mamille2/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (3,6,13,27,48) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr0/home/mamille2/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (9,10,27,42,43) have mixed types. Specify dtyp

67043234

In [13]:
with open(os.path.join(data_dirpath, 'blognames.txt'), 'w') as f:
    for blogname in blognames:
        f.write(f'{blogname}\n')

In [19]:
import random

# Random sample of 500 unique blog names, weighted by probability in data

selection = set()

while len(selection) < 1000:
    choice = random.choice(blognames)
    if not choice in selection:
        selection.add(choice)
        
len(selection)

1000

In [18]:
sorted(selection)[:20]

['123-tamarindo',
 '1998life-dream',
 '260to120thinspo',
 '666miss-u',
 'abrandarei',
 'absolutelynothingjustforgetit',
 'adultvideosclub',
 'alejandrard',
 'alexanderuhoton',
 'alexsorange',
 'aliceisinlesbianwonderlandagain',
 'alienpicnic',
 'allo-mishamigos',
 'allthingseurope',
 'alotofshittypeople',
 'amargedom',
 'amazinglybeautifulphotography',
 'amber2horny',
 'ambiguities',
 'amy-ambrosio']